<a href="https://colab.research.google.com/github/DownOnCoffee/GenAI_Learning/blob/main/movieReviewClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
data_path = '/content/IMDB Dataset.csv'
df = pd.read_csv(data_path)
df['review'][0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

In [ ]:
df= df.iloc[:10000]
df.drop_duplicates(inplace=True)

BASIC DATA PREPROCESSING STEPS

In [ ]:
import re
import nltk
import string
from nltk.corpus import stopwords
nltk.download("stopwords")
stop_words = set(stopwords.words("english"))   # use set for speed

puncs_chars = string.punctuation
df['review']=df['review'].str.lower()

def remove_html_tags(text):
    pattern = re.compile('<.*?>') #Every substring in text that matches the pattern is replaced with an empty string ''.
    return pattern.sub(r'', text)

def remove_puncs(text):
  for char in puncs_chars:
    text = text.replace(char, '')
  return text

def remove_stopwords(text):
    return " ".join([word for word in text.split() if word not in stop_words])

df["review"] = df["review"].apply(remove_html_tags)\
                           .apply(remove_puncs)\
                           .apply(remove_stopwords)

df['review'][0]


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


'one reviewers mentioned watching 1 oz episode youll hooked right exactly happened methe first thing struck oz brutality unflinching scenes violence set right word go trust show faint hearted timid show pulls punches regards drugs sex violence hardcore classic use wordit called oz nickname given oswald maximum security state penitentary focuses mainly emerald city experimental section prison cells glass fronts face inwards privacy high agenda em city home manyaryans muslims gangstas latinos christians italians irish moreso scuffles death stares dodgy dealings shady agreements never far awayi would say main appeal show due fact goes shows wouldnt dare forget pretty pictures painted mainstream audiences forget charm forget romanceoz doesnt mess around first episode ever saw struck nasty surreal couldnt say ready watched developed taste oz got accustomed high levels graphic violence violence injustice crooked guards wholl sold nickel inmates wholl kill order get away well mannered middle 

In [ ]:
#df.iloc[row_selection, column_selection]
#declaring inputs / outputs
X=df.iloc[:,0:1]
Y=df.iloc[:,1]

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
Y = encoder.fit_transform(Y)
Y

array([1, 1, 1, ..., 0, 0, 1])

In [ ]:
!pip install gensim

In [ ]:
from gensim.utils import simple_preprocess
from gensim.models import Word2Vec
tokens = X['review'].apply(simple_preprocess)

embedding_model = Word2Vec(
    window=10,min_count=2
)

embedding_model.build_vocab(tokens)
embedding_model.train(tokens,total_examples=embedding_model.corpus_count, epochs=embedding_model.epochs)



(5473405, 5901370)

In [ ]:
# Converting the tokens/words list to vectors
import numpy as np
X_w2v = []
for review in tokens:
    word_vectors = []
    for word in review:
        if word in embedding_model.wv:
            word_vectors.append(embedding_model.wv[word])
    if len(word_vectors) == 0:
        review_vector = np.zeros(embedding_model.vector_size)
    else:
        review_vector = np.mean(word_vectors, axis=0)
    X_w2v.append(review_vector)
X_w2v = np.array(X_w2v)

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X_w2v,Y,test_size=0.2,random_state=1)

In [ ]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(X_train,Y_train)

GaussianNB()

In [ ]:
Y_predict = model.predict(X_test)
from sklearn.metrics import accuracy_score,confusion_matrix
score = accuracy_score(Y_test, Y_predict)
score

0.7421131697546319